In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sjögrens_Syndrome"
cohort = "GSE93683"

# Input paths
in_trait_dir = "../../input/GEO/Sjögrens_Syndrome"
in_cohort_dir = "../../input/GEO/Sjögrens_Syndrome/GSE93683"

# Output paths
out_data_file = "../../output/preprocess/Sjögrens_Syndrome/GSE93683.csv"
out_gene_data_file = "../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE93683.csv"
out_clinical_data_file = "../../output/preprocess/Sjögrens_Syndrome/clinical_data/GSE93683.csv"
json_path = "../../output/preprocess/Sjögrens_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"CD8 T-cells from pSS patients and human healthy volunteers"
!Series_summary	"Multi-omics study was conducted to elucidate the crucial molecular mechanisms of primary Sjögren’s syndrome (SS) pathology. We generated multiple data set from well-defined patients with SS, which includes whole-blood transcriptomes, serum proteomes and peripheral immunophenotyping. Based on our newly generated data, we performed an extensive bioinformatic investigation. Our integrative analysis identified SS gene signatures (SGS) dysregulated in widespread omics layers, including epigenomes, mRNAs and proteins. SGS predominantly involved the interferon signature and ADAMs substrates. Besides, SGS was significantly overlapped with SS-causing genes indicated by a genome-wide association study and expression trait loci analyses. Combining the molecular signatures with immunophenotypic profiles revealed that cytotoxic CD8 ­T cells­ were associated with SGS. Further, we obser

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background info, we see it mentions "genome-wide transcriptome arrays"
# This suggests gene expression data should be available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Sjögren's Syndrome), row 0 contains 'disease state: HC' and 'disease state: pSS'
trait_row = 0

# For age, no information found in sample characteristics
age_row = None

# For gender, row 3 contains 'gender: Female' but it appears to be constant
# Since the sample characteristics only shows one value "Female" for gender
# and lacks variation, we consider gender as not available for our analysis
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert trait values to binary format (0: control, 1: Sjögren's Syndrome)
    """
    if not isinstance(value, str):
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if value.lower() == 'hc' or value.lower() == 'healthy control':
        return 0
    elif value.lower() == 'pss' or value.lower() == 'ss':
        return 1
    return None

def convert_age(value):
    """
    Convert age values to continuous format
    """
    # Not used since age data isn't available
    return None

def convert_gender(value):
    """
    Convert gender values to binary format (0: female, 1: male)
    """
    # Not used since gender data shows only one value (female)
    return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We have trait data, so extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_df)
    print("Preview of clinical features:")
    print(preview)
    
    # Save clinical data to CSV
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM2460433': [0.0], 'GSM2460434': [0.0], 'GSM2460435': [0.0], 'GSM2460436': [0.0], 'GSM2460437': [0.0], 'GSM2460438': [0.0], 'GSM2460439': [0.0], 'GSM2460440': [0.0], 'GSM2460441': [0.0], 'GSM2460442': [0.0], 'GSM2460443': [0.0], 'GSM2460444': [0.0], 'GSM2460445': [0.0], 'GSM2460446': [0.0], 'GSM2460447': [0.0], 'GSM2460448': [0.0], 'GSM2460449': [0.0], 'GSM2460450': [0.0], 'GSM2460451': [0.0], 'GSM2460452': [0.0], 'GSM2460453': [0.0], 'GSM2460454': [0.0], 'GSM2460455': [0.0], 'GSM2460456': [0.0], 'GSM2460457': [1.0], 'GSM2460458': [1.0], 'GSM2460459': [1.0], 'GSM2460460': [1.0], 'GSM2460461': [1.0], 'GSM2460462': [1.0], 'GSM2460463': [1.0], 'GSM2460464': [1.0], 'GSM2460465': [1.0], 'GSM2460466': [1.0], 'GSM2460467': [1.0], 'GSM2460468': [1.0], 'GSM2460469': [1.0], 'GSM2460470': [1.0], 'GSM2460471': [1.0], 'GSM2460472': [1.0], 'GSM2460473': [1.0], 'GSM2460474': [1.0], 'GSM2460475': [1.0], 'GSM2460476': [1.0], 'GSM2460477': [1.0], 'GSM2460478': [1.0], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous output
# These appear to be Affymetrix probe IDs (e.g., '1007_s_at', '1053_at')
# which are not standard human gene symbols and will need to be mapped

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns containing probe IDs and gene symbols
# From the gene annotation preview, we can see:
# - The 'ID' column contains the probe identifiers (matching what we see in gene_data)
# - The 'Gene Symbol' column contains the corresponding gene symbols

# 2. Get the gene mapping dataframe with the probe ID and gene symbol columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first 10 gene symbols to verify the mapping
print("Mapped gene expression data - first 10 genes:")
print(gene_data.index[:10])


Mapped gene expression data - first 10 genes:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data shape: {clinical_df.shape}")
print(clinical_df.head())

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from CD4 T-cells of pSS patients and healthy controls."
)

# 7. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (19845, 48)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Normalized gene data saved to ../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE93683.csv
Loaded clinical data shape: (1, 48)
                   GSM2460433  GSM2460434  GSM2460435  GSM2460436  GSM2460437  \
Sjögrens_Syndrome         0.0         0.0         0.0         0.0         0.0   

                   GSM2460438  GSM2460439  GSM2460440  GSM2460441  GSM2460442  \
Sjögrens_Syndrome         0.0         0.0         0.0         0.0         0.0   

                   ...  GSM2460471  GSM2460472  GSM2460473  GSM2460474  \
Sjögrens_Syndrome  ...         1.0         1.0         1.0         1.0   

                   GSM2460475  GSM2460476  GSM2460477  GSM2460478  GSM2460479  \
Sjögrens_Syndrome         1.0         1.0         1.0         1.0         1.0   

                   GSM2460480  
Sjögrens_Syndrome         1.0  

[1 rows x 48 columns]
Linked data shape: (48, 19846)
            Sjögrens_Syndrome      A1BG  A1BG-AS1      A1CF        A2M  \
GSM2460433                0.0  7.571450

Shape after handling missing values: (48, 19846)
For the feature 'Sjögrens_Syndrome', the least common label is '0.0' with 24 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Sjögrens_Syndrome' in this dataset is fine.



Linked data saved to ../../output/preprocess/Sjögrens_Syndrome/GSE93683.csv
